In [1]:
!pip install SoccerNet

In [1]:
import os
print(os.getcwd())

D:\dfl-bundesliga-data-shootout


In [2]:
os.chdir('D:/dfl-bundesliga-data-shootout')
print(os.getcwd())

D:\dfl-bundesliga-data-shootout


In [3]:
import SoccerNet
from SoccerNet.Downloader import SoccerNetDownloader

In [4]:
dfl_base = 'D:\dfl-bundesliga-data-shootout'
yolo_base = os.path.join(dfl_base,'yolov7')
yolo_base = yolo_base.replace('\\','/')
sn_track_base = os.path.join(dfl_base,'soccerNet/tracking')
sn_track_base = sn_track_base.replace('\\','/')
print(yolo_base)
print(sn_track_base)

D:/dfl-bundesliga-data-shootout/yolov7
D:/dfl-bundesliga-data-shootout/soccerNet/tracking


In [128]:
!mkdir -p $sn_track_base
!mkdir -p $yolo_base
!echo $sn_track_base/test.zip
!echo $sn_track_base/challenge.zip

mySoccerNetDownloader=SoccerNetDownloader(LocalDirectory="/soccerNet/tracking")
mySoccerNetDownloader.downloadDataTask(task="tracking", split=["train", "test", "challenge"])
#window에서는 수작업
!unzip -q -d $sn_track_base $sn_track_base/train.zip
!git clone https://github.com/WongKinYiu/yolov7 
!cd $yolo_base && wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6_training.pt

명령 구문이 올바르지 않습니다.
명령 구문이 올바르지 않습니다.


D:/dfl-bundesliga-data-shootout/soccerNet/tracking/test.zip
D:/dfl-bundesliga-data-shootout/soccerNet/tracking/challenge.zip


FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: '/soccerNet/tracking\\tracking\\train.zip'

In [5]:
from glob import glob
train_dirs = sorted(glob(f'{sn_track_base}/train/train/SNMOT*'))
#tst_dirs = sorted(glob(f'{sn_track_base}/test/SNMOT*'))
test_dirs = train_dirs[50:]
train_dirs = train_dirs[:50]

debug=True
if debug:
    train_dirs = train_dirs[:5]
    test_dirs = test_dirs[:5]

split_dirs = {
    'train':train_dirs,
    'valid':test_dirs
}

# ４ types of labels are used.
labels = ['ball', 'player', 'referee', 'goalkeepers']
label_dict = {'ball': 0, 'player': 1, 'referee': 2, 'goalkeeper': 3, 'goalkeepers': 3}
print(train_dirs)

['D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train\\SNMOT-060', 'D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train\\SNMOT-061', 'D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train\\SNMOT-062', 'D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train\\SNMOT-063', 'D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train\\SNMOT-064']


In [6]:
import os
#(1) image file path
yolo_train_img_dir = f'{yolo_base}/images/train'
yolo_valid_img_dir = f'{yolo_base}/images/valid'

#(2) label file path
yolo_train_label_dir = f'{yolo_base}/labels/train'
yolo_valid_label_dir = f'{yolo_base}/labels/valid'

#(3) config file path
yaml_file = f'{yolo_base}/data.yaml'
#!rm -rf /home/tito/kaggle/dfl-bundesliga-data-shootout/work/yolov5
os.makedirs(yolo_train_img_dir, exist_ok=True)
os.makedirs(yolo_valid_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True)
os.makedirs(yolo_valid_label_dir, exist_ok=True)
print(yolo_train_img_dir)
print(yolo_valid_img_dir)

print(yolo_train_label_dir)
print(yolo_valid_label_dir)

print(yaml_file)

D:/dfl-bundesliga-data-shootout/yolov7/images/train
D:/dfl-bundesliga-data-shootout/yolov7/images/valid
D:/dfl-bundesliga-data-shootout/yolov7/labels/train
D:/dfl-bundesliga-data-shootout/yolov7/labels/valid
D:/dfl-bundesliga-data-shootout/yolov7/data.yaml


In [7]:
!pip install opencv-python

In [7]:
import re
import cv2
import pandas as pd
import numpy as np
# convert from x,y,w,h to yolo format
def get_yolo_format_bbox(img_w, img_h, box):
    w = box[2]
    h = box[3]
    xc = box[0] + int(np.round(w/2))
    yc = box[1] + int(np.round(h/2))
    box = [xc/img_w, yc/img_h, w/img_w, h/img_h]
    box = [f"{i:.4g}" for i in box]
    return box
    
# get SoccerNet label info 
def get_info(info):
    results = []
    for line in open(info):
        m = re.match('trackletID_(\d+)= (\S*).*', line.replace(';', ' '))
        if m:
            if m.group(2) not in label_dict:
                #print('bad label:', m.group(2))
                continue 
            results.append([m.group(1), m.group(2)])
    return pd.DataFrame(results, columns=['id','lbl']).set_index('id').to_dict()['lbl']

# make image simlink and label files
for split in split_dirs:
    print(split)
    if split == 'train':
        yolo_img_dir = yolo_train_img_dir
        yolo_label_dir = yolo_train_label_dir
    else:
        yolo_img_dir = yolo_valid_img_dir
        yolo_label_dir = yolo_valid_label_dir
    for this_dir in split_dirs[split]:
        this_dir = this_dir.replace('\\','/')
        print('this_dir:',this_dir)
        video = this_dir.split('/')[-1]
        info = this_dir + '/gameinfo.ini'
        det = this_dir + '/gt/gt.txt'
        info_dict = get_info(info)
        det_df = pd.read_csv(det, names=['frame','player','x','y','w','h','f1','f2','f3','f4'], usecols=['frame','player','x','y','w','h'])
        det_df.head(5)
        det_df['label'] = det_df.player.astype(str).map(info_dict)
        det_df['label_id'] = det_df['label'].map(label_dict)
        # check
        ng_list = list(det_df[det_df.label_id.isnull()].label.unique())
        if len(ng_list)>0:
            #print('ng_list:',ng_list, det_df.dropna().shape, det_df.shape)
            det_df = det_df.dropna()
        for grp, grp_df in det_df.groupby('frame'):
            frame = f'{grp:06}'
            img_file = f'{this_dir}/img1/{frame}.jpg'
            dst_file = f'{yolo_img_dir}/{video}_{frame}.jpg'
            if not os.path.exists(dst_file):
                os.symlink(img_file, dst_file)
                #print(img_file)
            img = cv2.imread(dst_file)
            height, width, _ = img.shape 
            bboxes = []
            for arr in grp_df[['x', 'y', 'w', 'h', 'label_id']].values:
                box = get_yolo_format_bbox(width, height, arr[:4])
                bboxes.append([arr[4]]+box)
            file_name = f'{yolo_label_dir}/{video}_{frame}.txt'
            with open(file_name, 'w') as f:
                for i, bbox in enumerate(bboxes):
                    bbox = [str(i) for i in bbox]
                    str_bbox = ' '.join(bbox)
                    f.write(str_bbox)
                    f.write('\n')

train
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-060
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-061
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-062
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-063
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-064
valid
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-164
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-165
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-166
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-167
this_dir: D:/dfl-bundesliga-data-shootout/soccerNet/tracking/train/train/SNMOT-168


In [8]:
import yaml
# Dump config file
data_yaml = dict(
    train = yolo_train_img_dir,
    val = yolo_valid_img_dir,
    nc = 4,
    names = labels
)

with open(yaml_file, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [39]:
!pip install tensorboard

  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)


In [10]:
os.chdir('D:/dfl-bundesliga-data-shootout/yolov7')
print(os.getcwd())
!python train_aux.py \
    --workers 0 \
    --batch-size 8 \
    --data data.yaml \
    --img 1280 \
    --epochs 10 \
    --cfg cfg/training/yolov7-w6.yaml \
    --weights yolov7-w6_training.pt \
    --name yolov7-w6-custom \
    --hyp data/hyp.scratch.custom.yaml

D:\dfl-bundesliga-data-shootout\yolov7
^C


In [21]:
# predict!
check_images = f'{test_dirs[0]}/img1'
weight_file = 'runs/train/yolov7-w6-custom/weights/best.pt'
!cd $yolo_base && python detect.py \
    --weights $weight_file \
    --source  $check_images \
    --img-size 1280 \
    --project dfl

IndexError: list index out of range

In [ ]:
# Check some results. 
display(Image(f'{yolo_base}/dfl/exp/000200.jpg'))
display(Image(f'{yolo_base}/dfl/exp/000400.jpg'))
display(Image(f'{yolo_base}/dfl/exp/000600.jpg'))